In [3]:
%run importsandsuch.ipynb

Path to dataset files: /Users/axelvandenheuvel/.cache/kagglehub/datasets/robbypeery/college-basketball-pbp-23-24/versions/18
(10858, 34)
              precision    recall  f1-score   support

           0       0.99      0.79      0.88      2116
           1       0.06      0.71      0.12        42

    accuracy                           0.79      2158
   macro avg       0.53      0.75      0.50      2158
weighted avg       0.97      0.79      0.87      2158



In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 1) Define which features to use and your window size
feature_cols = ['team_encoded', 'event_encoded', 'time_seconds', 'points_scored']
window_size = 5   # you said 3–5 plays; here we use 5

# 2) Build sequences (X) and labels (y)
X_seq, y_seq = [], []
for i in range(window_size, len(df)):
    # skip windows spanning multiple games
    if df.iloc[i-window_size:i]['game_id'].nunique() > 1:
        continue
    X_seq.append(df.iloc[i-window_size:i][feature_cols].values)
    y_seq.append(df.iloc[i]['run_start'])

X = np.array(X_seq)     # shape: (n_samples, 5, 4)
y = np.array(y_seq)     # shape: (n_samples,)

print("X.shape:", X.shape, " y.shape:", y.shape)

# 3) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 4) Build a simple LSTM model
model = Sequential([
    LSTM(64, input_shape=(window_size, len(feature_cols))),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=1e-3),
    metrics=['accuracy']
)

model.summary()

# 5) Train
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64
)

# 6) Evaluate on the hold-out set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

print(classification_report(y_test, y_pred))


X.shape: (10721, 5, 4)  y.shape: (10721,)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,777 (77.25 KB)

 Trainable params: 19,777 (77.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8745 - loss: 0.3147 - val_accuracy: 0.9808 - val_loss: 0.0972
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9793 - loss: 0.1093 - val_accuracy: 0.9808 - val_loss: 0.0991
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9812 - loss: 0.1029 - val_accuracy: 0.9808 - val_loss: 0.0977
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9808 - loss: 0.0997 - val_accuracy: 0.9808 - val_loss: 0.0973
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9814 - loss: 0.0957 - val_accuracy: 0.9808 - val_loss: 0.0955
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9810 - loss: 0.0982 - val_accuracy: 0.9808 - val_loss: 0.0970
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9815 - loss: 0.0948 - val_accuracy: 0.9808 - val_loss: 0.0969
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9833 - loss: 0.0876 - val_accuracy: 0.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  